 Manual Model Explainability Engine


In [27]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()

In [4]:
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


In [6]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [7]:

model = LogisticRegression(max_iter=500)
model.fit(X_train_scaled, y_train)

print("Accuracy:", accuracy_score(y_test, model.predict(X_test_scaled)))

Accuracy: 0.9736842105263158


In [24]:
feature_names = X.columns
weights = model.coef_[0]         # shape: (num_features,)
intercept = model.intercept_[0]

def explain_prediction(x_scaled):
    """
    x_scaled: 1D numpy array of scaled features
    returns dataframe of contributions
    """

    # raw contributions
    contributions = x_scaled * weights

    df = pd.DataFrame({
        "feature": feature_names,
        "scaled_value": x_scaled,
        "weight": weights,
        "raw_contribution": contributions
    })

    # total logit
    logit = intercept + contributions.sum()

    # convert logit to probability
    prob = 1 / (1 + np.exp(-logit))

    # normalize contributions (absolute percentage of total)
    total_abs = np.sum(np.abs(contributions))
    df["normalized_importance"] = np.abs(contributions) / total_abs

    # sort by impact
    df = df.sort_values("normalized_importance", ascending=False)

    return df, logit, prob


In [26]:
sample_idx = 5
x = X_test_scaled[sample_idx]

explain_df, logit, prob = explain_prediction(x)

print("\nPredicted class probability:", prob)
print("\nTop feature contributions:\n")
print(explain_df.head(10))



Predicted class probability: 9.480877729723756e-11

Top feature contributions:

                 feature  scaled_value    weight  raw_contribution  \
21         worst texture      2.291730 -1.350606         -3.095223   
7    mean concave points      2.728159 -1.119804         -3.055004   
26       worst concavity      3.174988 -0.943053         -2.994182   
6         mean concavity      3.306880 -0.801458         -2.650326   
28        worst symmetry      1.873723 -1.208200         -2.263833   
27  worst concave points      2.311233 -0.778217         -1.798641   
5       mean compactness      3.307983  0.540164          1.786853   
20          worst radius      1.977724 -0.879840         -1.740081   
10          radius error      1.145429 -1.268178         -1.452608   
23            worst area      1.664783 -0.841846         -1.401491   

    normalized_importance  
21               0.093251  
7                0.092040  
26               0.090207  
6                0.079848  
28      